# Library

In [38]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random

# Data Load & Preprocessing
- 훈련에 필요없는 index 컬럼 삭제.
- missing value를 모두 NAN 문자열로 대체
- dtype object 인 컬럼들을 onehot encoding

In [39]:
train = pd.read_csv('dataset/train.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 


test = pd.read_csv('dataset/test.csv')
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

submit = pd.read_csv('dataset/sample_submission.csv')

In [41]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         10000 non-null  object 
 1   car            10000 non-null  object 
 2   reality        10000 non-null  object 
 3   child_num      10000 non-null  int64  
 4   income_total   10000 non-null  float64
 5   income_type    10000 non-null  object 
 6   edu_type       10000 non-null  object 
 7   family_type    10000 non-null  object 
 8   house_type     10000 non-null  object 
 9   DAYS_BIRTH     10000 non-null  int64  
 10  DAYS_EMPLOYED  10000 non-null  int64  
 11  FLAG_MOBIL     10000 non-null  int64  
 12  work_phone     10000 non-null  int64  
 13  phone          10000 non-null  int64  
 14  email          10000 non-null  int64  
 15  occyp_type     10000 non-null  object 
 16  family_size    10000 non-null  float64
 17  begin_month    10000 non-null  float64
dtypes: floa

In [31]:
train.dtypes

gender            object
car               object
reality           object
child_num          int64
income_total     float64
income_type       object
edu_type          object
family_type       object
house_type        object
DAYS_BIRTH         int64
DAYS_EMPLOYED      int64
FLAG_MOBIL         int64
work_phone         int64
phone              int64
email              int64
occyp_type        object
family_size      float64
begin_month      float64
credit           float64
dtype: object

In [32]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [33]:
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)
train.dtypes

child_num                                   int64
income_total                              float64
DAYS_BIRTH                                  int64
DAYS_EMPLOYED                               int64
FLAG_MOBIL                                  int64
work_phone                                  int64
phone                                       int64
email                                       int64
family_size                               float64
begin_month                               float64
credit                                    float64
gender_F                                  float64
gender_M                                  float64
car_N                                     float64
car_Y                                     float64
reality_N                                 float64
reality_Y                                 float64
income_type_Commercial associate          float64
income_type_Pensioner                     float64
income_type_State servant                 float64


In [34]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)
test.dtypes

child_num                                   int64
income_total                              float64
DAYS_BIRTH                                  int64
DAYS_EMPLOYED                               int64
FLAG_MOBIL                                  int64
work_phone                                  int64
phone                                       int64
email                                       int64
family_size                               float64
begin_month                               float64
gender_F                                  float64
gender_M                                  float64
car_N                                     float64
car_Y                                     float64
reality_N                                 float64
reality_Y                                 float64
income_type_Commercial associate          float64
income_type_Pensioner                     float64
income_type_State servant                 float64
income_type_Student                       float64


In [35]:
train_index=train[train['child_num']>=7].index
train.drop(train_index, inplace=True)
submit.drop(train_index, inplace=True)

KeyError: '[10731 25313 25390 25638] not found in axis'

# Training
- 데이터 분리는 StratifiedKFold 를 사용하여 y값 분포를 비슷하게 분리시킴. -> 5-fold
- lightgbm의 default parameter로 훈련.
- 30번 이상 개선 없을 경우 중단.
- 각 5개의 fold를 훈련하여 저장

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [ ]:
params={}
params['boosting_type']='gbdt'
params['learning_rate']='0.01'
params['max_depth']='16'
params['n_estimators']='1000'
params['objective']='regression'
params['metric']='multi_logloss'
params['is_training_metric']='True'
params['num_leaves']='31' #중요
params['feature_fraction']='0.7'#몇프로feature랜덤하게 학습 열 샘플링
params['bagging_fraction']='0.7'#행 샘플링
params['seed']='2020'
params['early_stopping_round']='50'
params['min_data_in_leaf']='20' #중요



In [ ]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values
    
    lgb = LGBMClassifier(boosting_type='gbdt',
                        learning_rate=0.1,
                         max_depth=30,
                         n_estimators=1500,
                         objective='regression',
                         metric='multi_logloss',
                         is_training_metric=True,
                         num_leaves=150, #중요
                         feature_fraction=0.7,#몇프로feature랜덤하게 학습 열 샘플링
                         bagging_fraction=0.7, #행 샘플링
                         seed=2020,
                         early_stopping_round=100,
                         min_data_in_leaf=5, #중요
                         tree_learner='feature',
                         extra_trees='False'
                        )
    lgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], early_stopping_rounds=50,
           verbose=100) #verbose 횟수마다 출력
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')


# Test inference
- 각 fold를 훈련시킨 lightgbm model로 predict.
- 해당 대회는 logloss score를 겨루는 것이기 때문에 각 class의 probability를 얻어야함.
- 대부분의 머신러닝 모델에서 predict, predict_proba를 구분하여 사용함.
- predict는 class 출력을 해주고 predict_proba는 class별 probability를 출력해줌.
- predict_proba를 사용하여 예측한 것을 5-fold 더하여 평균내어 앙상블.

In [ ]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

In [ ]:
submit.to_csv('submit/20210503_test_submit_ensemble.csv', index=False) # 0.7272812144

In [ ]:
submit.head(20)